In [3]:
import pandas as pd
import numpy as np
import dknlab_tools
import re

import csv
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats
import colorcet as cc

import statsmodels.api as sm

import bokeh.io
import bokeh.plotting
import bokeh.palettes
import bokeh.models
from bokeh.transform import jitter, factor_cmap

import tqdm
import re
import glob

import random
random.seed(42)

bokeh.io.output_notebook()

import warnings
warnings.filterwarnings('ignore')

Loading BokehJS ...

In [4]:
maps = sorted(glob.glob('../data/platereader/2020_08_03*map.*'))
data = sorted(glob.glob('../data/platereader/2020_08_03*fluor.*'))

In [5]:
maps

['../data/platereader/2020_08_03_PCAoxidation_plate_condition_map.xlsx']

In [6]:
dfs = []

for d, m in zip(data, maps):
    date = re.search(r'[0-9]+_[0-9]+_[0-9]+', d).group()
    
    data_df = pd.read_excel(d, skiprows=1)
    data_df = data_df.melt(id_vars='Time [hr]', var_name='Well', value_name='PCAred fluorescence (AU)')
    
    data_df['Time [hr]'] = data_df['Time [hr]'].astype('str').apply(dknlab_tools.biotek._str2hours)
    
    map_df = dknlab_tools.biotek.plate_condmap(m).reset_index()
    df = data_df.merge(map_df, on='Well')
    df = df.dropna()
    df['date'] = date
    
    # average the technical replicates
    
    mean_df = df.groupby(['date', 'Strain', 'Condition', 'Time [hr]', 'Condition Conc. (µM)']).agg('mean').reset_index()
    
    dfs.append(mean_df)

In [7]:
dfs[0]

,date,Strain,Condition,Time [hr],Condition Conc. (µM),PCAred fluorescence (AU)
0,2020_08_03,Abiotic,PCA,0.061,200,5917.000000
1,2020_08_03,Abiotic,PCA,0.144,200,5917.666667
2,2020_08_03,Abiotic,PCA,0.228,200,5903.000000
3,2020_08_03,Abiotic,PCA,0.311,200,5942.000000
4,2020_08_03,Abiotic,PCA,0.394,200,5926.333333
...,...,...,...,...,...,...
21959,2020_08_03,calibration,PCA,24.061,3.125,916.000000
21960,2020_08_03,calibration,PCA,24.061,400,8045.000000
21961,2020_08_03,calibration,PCA,24.061,50,2874.000000
21962,2020_08_03,calibration,PCA,24.061,6.25,1044.000000


In [8]:
all_data = pd.concat(dfs)

### Fluorescence calibration functions

In [9]:
def general_hill(x, Ka, n, A, B, C):
    """
    Use a generalized hill function for the calibration curve.
    """
    
    y = B + A / (C + (Ka / x) ** n)
    
    return y

def inverse_general_hill(y, Ka, n, A, B, C):
    """
    Inverse function for deriving concentrations from fluorescence values.
    """
    
    x = Ka / ((A / (y - B) - C) ** (1/n))
    
    return x

def get_calib_data(fluor_df):
    """
    Function to extract calibration data from the general dataframe(s).
    """
    
    calib_df = fluor_df.loc[fluor_df['Strain'] == 'calibration']
    calib_df['Condition Conc. (µM)'] = calib_df['Condition Conc. (µM)'].astype(float)
    
    return calib_df

def plot_calib_point(calib_df, title=None, alpha=0.05):
    """
    Plotter for the calibration data.
    """
    
    
    fig = bokeh.plotting.figure(height=400, 
                                width=600, 
                                title=title, 
                                x_axis_label='µM PCAred', 
                                y_axis_label='Fluorescence (AU)')
    
    c = fig.circle(calib_df['Condition Conc. (µM)'].values, 
                   calib_df['PCAred fluorescence (AU)'].values, 
                   size=5, 
                   alpha=alpha, 
                   legend='Calibration measurements')
    
    fig.legend.location = 'bottom_right'
    
    return fig
    
def fit_hill(calib_df, alpha=0.05, show_plot=False, p0=[150, 2, 9000, 1000, 1]):
    """
    Function to fit the generalized Hill function to the calibration data.
    """
    
    xdata = calib_df['Condition Conc. (µM)'].values
    ydata = calib_df['PCAred fluorescence (AU)'].values
    
    popt, pcov = scipy.optimize.curve_fit(general_hill, xdata, ydata, p0=p0)
    
    plot = plot_calib_point(calib_df, title='Fit of calibration model', alpha=alpha)
    
    x = np.linspace(0, 800, 100)
    fit = general_hill(x, *popt)
    
    plot.line(x, fit, color='black')
    
    if show_plot:
        bokeh.io.show(plot)
    
    return popt, pcov

def convert_fluor_to_conc(fluor_exp_df, popt):
    """
    Function to convert fluorescence measurements to concentrations.
    """
    
    fluor_exp_df['measured PCAred (µM)'] = [inverse_general_hill(f, *popt) for f in fluor_exp_df['PCAred fluorescence (AU)']]
    
    return fluor_exp_df

def fitting_pipeline(df, p0=[150, 2, 9000, 1000, 1]):
    """
    Function to bring above utilities together.
    """
    
    calib_df = get_calib_data(df)
    exp_df = df.loc[df['Strain'] != 'calibration']
    
    popt, pcov = fit_hill(calib_df, p0)
    
    exp_df = convert_fluor_to_conc(exp_df, popt)
    
    return exp_df

### LOWESS fitting functons

In [10]:
def lowess_fit(single_data_df, timestep=5/60, window=24, plot=True):
    """
    Function to perform a LOWESS fit of PCA oxidation data.
    
    Params:
    -------
    single_data_df : pandas DataFrame
        The data for a single strain under a single condition from a single experiment.
        This is either a biological replicate from a single well or the mean of
        technical replicates from a given day.
        
    timestep : float, default 5/60
        The interval (in hours) between measurements in the timeseries
        
    window : int, default 24
        The number of time points oover which the LOWESS algorith should smooth.
        If the timestep is 5/60, a window of 24 corresponds to 2 hours
        
    plot : Bool, default True
        Should this function plot the data and fit?
    """
    
    x = single_data_df['Time [hr]'].values
    y = single_data_df['measured PCAred (µM)'].values
    
    smoothed = sm.nonparametric.lowess(
        exog=x, 
        endog=y,
        is_sorted=False,
        return_sorted=True,
        frac=window/len(x)
    )
    
    if plot:
        lowess_fit_plot = bokeh.plotting.figure(width=600, height=400)
        
        lowess_fit_plot.circle(x=x, y=y)
        lowess_fit_plot.line(smoothed[:,0], smoothed[:,1], line_width=2, color='red')
        
        bokeh.io.show(lowess_fit_plot)
    
    return smoothed[:,0], smoothed[:,1]

def get_lowess_derivative(lowess_x, lowess_y, timestep=5/60, plot=True):
    """
    Function to calculate the derivative of a LOWESS fit of PCA oxidation data.
    
    Params:
    -------
    lowess_x : numpy array
        The x-axis (time points) of the lowess fit
        
    lowess_y : numpy array
        The y-axis (PCAred concentration) of the lowess fit
        
    timestep : float, default 5/60
        The interval (in hours) between measurements in the timeseries
        
    plot : Bool, default True
        Should this function plot the data and fit?
    """
    
    derivative = np.gradient(lowess_y, timestep)
    
    if plot:
        lowess_derivative_plot = bokeh.plotting.figure(width=600, height=400)
        
        lowess_derivative_plot.line(lowess_x, derivative)
        
        bokeh.io.show(lowess_derivative_plot)
    
    return derivative

def get_max_oxidation_rate_from_lowess_fit(single_data_df, timestep=5/60, window=28, plot=True):
    """
    Function to perform a LOWESS fit of PCA oxidation data.
    
    Params:
    -------
    single_data_df : pandas DataFrame
        The data for a single strain under a single condition from a single experiment.
        This is either a biological replicate from a single well or the mean of
        technical replicates from a given day.
        
    timestep : float, default 5/60
        The interval (in hours) between measurements in the timeseries
        
    window : int, default 28
        The number of time points oover which the LOWESS algorith should smooth.
        If the timestep is 5/60, a window of 28 corresponds to 140 minutes.
        This selection is enough to smooth over some erraticity in the plate reader
        but still matches what I initially did with scanning windows with size 2
        hours
        
    plot : Bool, default True
        Should this function plot the data and fit?
    """
    
    times = single_data_df['Time [hr]'].values
    
    interval = np.mean([x2-x1 for x2, x1 in zip(times[1:], times[:-1])])
    
    # window = 2.33 / interval
    
    lowess_x, lowess_y = lowess_fit(single_data_df, timestep=interval, window=window, plot=plot)
    
    derivative = get_lowess_derivative(lowess_x, lowess_y, timestep=interval, plot=plot)
    
    max_oxidation_rate = np.max(derivative * -1)
    
    return max_oxidation_rate

def get_time_to_thresh_from_lowess_fit(single_data_df, timestep=5/60, window=24, plot=True, thresh=0.5):
    """
    Function to perform a LOWESS fit of PCA oxidation data. Return the time
    to half maximum PCAred concentration from maximum PCAred concentration, if attained
    
    Params:
    -------
    single_data_df : pandas DataFrame
        The data for a single strain under a single condition from a single experiment.
        This is either a biological replicate from a single well or the mean of
        technical replicates from a given day.
        
    timestep : float, default 5/60
        The interval (in hours) between measurements in the timeseries
        
    window : int, default 24
        The number of time points oover which the LOWESS algorith should smooth.
        If the timestep is 5/60, a window of 24 corresponds to 2 hours
        
    plot : Bool, default True
        Should this function plot the data and fit?
    """
    
    lowess_x, lowess_y = lowess_fit(single_data_df, timestep=timestep, window=window, plot=plot)
    
    # There are some cases with fumarate when the final [PCAred]
    # concentration is higher than the max before the oxidation
    # begins. So, I can't just take the max: I have to take the
    # maximum that comes before the minimum
    
    index_min = list(lowess_y).index(np.min(lowess_y))
    
    left_part = lowess_y[:index_min]
    if len(left_part) > 0:
        index_max = list(lowess_y).index(np.max(lowess_y[:index_min]))
    else:
        index_max = list(lowess_y).index(np.max(lowess_y))
        
    less_than_thresh = lowess_y <= np.max(lowess_y) * thresh
    try:
        index_thresh = list(less_than_thresh).index(True)
    
    except:
        return None
    
    time_to_thresh_from_max = lowess_x[index_thresh] - lowess_x[index_max]
    
    return time_to_thresh_from_max

def get_lowess_df(calibrated_df, 
                  return_param='rate', 
                  grouping=['date', 'Strain', 'Condition', 'Medium'], 
                  cat_fwd=False, 
                  thresh=0.5,
                  window=24,
                 ):
    """
    Function to get a data frame for plotting max. PCA oxidation rates
    from LOWESS fitting of calibrated data.
    
    calibrated_df : pandas DataFrame
        The PCA oxidation data, calibrated by time point
    
    grouping : list, default ['date', 'Strain', 'Condition', 'Medium']
        The grouping to get unique experimental runs for data fitting
    
    cat_fwd : Bool, default True
        When True, the categories (which will be used for nested plotting by Bokeh),
        will be (Strain, Condition). When False, the categories will be (Condition, Strain)
    """
    
    
    condition_palette = bokeh.palettes.Colorblind4
    strain_palette_dict = bokeh.palettes.Category20
    
#     strains = sorted([s for s in calibrated_df['Strain'].unique() if not re.match(r'[0-9]', s[-1])])

    strains = []
    for s in calibrated_df['Strain'].unique():
        if re.match(r'[0-9]', s[-1]):
            s = s[:-1]
        
        if s not in strains:
            strains.append(s)
            
    strains = sorted(strains)
    
    strain_palette = strain_palette_dict[len(strains)]
        
    cmap = {
        'PCA': 'white',
        'PCA, DMSO': condition_palette[1],
        'PCA, NO3': condition_palette[0],
        'PCA, Fum': condition_palette[2],
        'PCA, TMAO': condition_palette[3]
    }
    
    grouped = calibrated_df.groupby(grouping)
    
    groups = []
    dates = []
    return_params = []
#     max_redox_rate_errors = []
    fill_colors = []
#     line_colors = []
    shapes = []
    strain_labels = []
    
    for grp in tqdm.tqdm(grouped):
        dates.append(grp[0][0])
        condition = grp[0][2]
        
        strain = grp[0][1]
        
        if strain[-1] in [str(x) for x in range(10)]:
            # biological replicate
#             line_colors.append('black')
            shapes.append('circle')
            # strip the number so that the replicates plot together
            strain = strain[:-1]
        else:
            #technical replicate
#             line_colors.append('red')
            shapes.append('square')
            
        if cat_fwd:
            groups.append((strain, condition))
            fill_colors.append(cmap[condition])
        else:
            groups.append((condition, strain))
            
            fc = strain_palette[strains.index(strain)]
            fill_colors.append(fc)
        
        strain_labels.append(strain)
        
        if return_param == 'rate':
            rp = get_max_oxidation_rate_from_lowess_fit(grp[1], plot=False, window=window)
        elif return_param == 'time':
            rp = get_time_to_thresh_from_lowess_fit(grp[1], plot=False, thresh=thresh, window=window)
        
        return_params.append(rp)
#         max_redox_rate_errors.append(max_err)

#     error_bars = []
#     for r, e in zip (max_redox_rates, max_redox_rate_errors):
#         error_bars.append([r*-1 - e, r*-1 + e])
        
    lowess_analysis_df = pd.DataFrame({'return param': np.array(return_params), 
#                                 'err': max_redox_rate_errors, 
#                                 'err_bars': error_bars,
                                'date': dates,
                                'cat': groups,
                                'fill color': fill_colors,
                                'shape': shapes,
                                'strain': strain_labels,
#                                 'line color': line_colors
                           })
    
    lowess_analysis_df = lowess_analysis_df.sort_values(by=['cat', 'strain'], key=lambda x: x.str.len())
    
    return lowess_analysis_df

def lowess_plot(lowess_plot_df, 
                show=True, 
                height=600, 
                width=1200, 
                jitter_width=0.6, 
                cat_fwd=True, 
                outfile=None, 
                y_axis_label='max PCA oxidation rate (µM/hr)',
                semilog=False
               ):
    
    lowess_bio_cds = bokeh.models.ColumnDataSource(lowess_plot_df.loc[lowess_plot_df['shape'] == 'circle'])
    lowess_tech_cds = bokeh.models.ColumnDataSource(lowess_plot_df.loc[lowess_plot_df['shape'] == 'square'])
    factors = bokeh.models.FactorRange(factors=lowess_plot_df['cat'].unique())

    TOOLTIPS = [
        ("date", "@date")
    ]

    if semilog:
        lowess_analysis_plot = bokeh.plotting.figure(
            x_range=factors,
        #     y_axis_type='log',
            height=height,
            width=width,
            tooltips=TOOLTIPS,
            y_axis_label = y_axis_label,
            y_axis_type = 'log'
        )

    else:
        lowess_analysis_plot = bokeh.plotting.figure(
            x_range=factors,
        #     y_axis_type='log',
            height=height,
            width=width,
            tooltips=TOOLTIPS,
            y_axis_label = y_axis_label
        )



    circle = bokeh.models.Scatter(
        y='return param', 
        x=jitter('cat', range=lowess_analysis_plot.x_range, width=jitter_width),
        fill_color='fill color', 
        line_color='black', 
        line_width=2, 
        size=10,
        line_alpha=0.7,
        marker='circle'
    )
    
    square = bokeh.models.Scatter(
        y='return param', 
        x=jitter('cat', range=lowess_analysis_plot.x_range, width=jitter_width),
        fill_color='fill color', 
        line_color='black', 
        line_width=2, 
        size=10,
        line_alpha=0.7,
        marker='square'
    )
    
    lowess_analysis_plot.add_glyph(lowess_bio_cds, circle)
    lowess_analysis_plot.add_glyph(lowess_tech_cds, square)
    
    bio_data = [j for j in lowess_bio_cds.data['return param'] if j]
    tech_data = [k for k in lowess_tech_cds.data['return param'] if k]
    
    if len(bio_data) == 0:
        lowess_analysis_plot.y_range.start = -0.1 * np.max(tech_data)
        
    elif len(tech_data) == 0:
        lowess_analysis_plot.y_range.start = -0.1 * np.max(bio_data)
        
    elif len(tech_data) + len(bio_data) == 0:
        lowess_analysis_plot.y_range.start = 0
        
    else:
        lowess_analysis_plot.y_range.start = np.min([-0.1 * np.max(bio_data), -0.1 * np.max(tech_data)])
    



    lowess_analysis_plot.xaxis.major_label_orientation = np.pi/4
    # box_plot.xaxis.group_label_orientation = np.pi/2
    lowess_analysis_plot.xgrid.grid_line_color = None
    lowess_analysis_plot.output_backend = 'svg'
    lowess_analysis_plot.yaxis.axis_label_text_font_size = '12pt'
    lowess_analysis_plot.xaxis.axis_label_text_font_size = '14pt'
    lowess_analysis_plot.yaxis.major_label_text_font_size = '10pt'
    lowess_analysis_plot.xaxis.major_label_text_font_size = '12pt'
    
    if show:
        bokeh.io.show(lowess_analysis_plot)
    
    if outfile:
        bokeh.io.export_svg(lowess_analysis_plot, filename=outfile)
        
    return lowess_analysis_plot

### Check overall calibration

In [12]:
test_calib_df = get_calib_data(df)
#     print(np.max(calib_df['Condition Conc. (µM)'].values))
    
    # print(date)
popt, pcov = fit_hill(test_calib_df, alpha=0.1, show_plot=True, p0=[150, 2, 9000, 1000, 1])

### Calibrate by timepoint

In [13]:
def calibrate_by_time_point(df):
    """
    Function to calibrate the data by each time point
    """
    grouped = df.groupby('Time [hr]')
    
    calibrated_dfs = []
    
    for grp in grouped:
        t = grp[0]
        mini_df = grp[1]
        
        mini_df = fitting_pipeline(mini_df)
        
        calibrated_dfs.append(mini_df)
        
    calibrated = pd.concat(calibrated_dfs)
    
    return calibrated

In [14]:
calibrated_by_time_point = calibrate_by_time_point(all_data)
calibrated_by_time_point.head()

,date,Strain,Condition,Time [hr],Condition Conc. (µM),PCAred fluorescence (AU),measured PCAred (µM)
0,2020_08_03,Abiotic,PCA,0.061,200,5917.000000,185.477357
289,2020_08_03,Abiotic,"PCA, Fum",0.061,"200, 10000",5803.000000,177.383106
578,2020_08_03,Abiotic,"PCA, NO2",0.061,"200, 10000",5143.333333,137.393783
867,2020_08_03,Abiotic,"PCA, NO3",0.061,"200, 10000",6028.000000,193.771226
1156,2020_08_03,E. coli1,PCA,0.061,200,5774.000000,175.388294


In [15]:
lowess_df_rate = get_lowess_df(
    calibrated_by_time_point, 
    return_param='rate', 
    grouping=['date', 'Strain', 'Condition'], 
    cat_fwd=False, 
    thresh=0.5,
    window=24
)

100%|███████████████████████████████████████████| 64/64 [00:02<00:00, 28.30it/s]


In [20]:
lowess_rate_plot = lowess_plot(lowess_df_rate, 
                show=True, 
                height=400, 
                width=800, 
                jitter_width=0.3, 
#                 cat_fwd=True, 
                outfile='./plots/figure16_rate.svg', 
                y_axis_label='max PCA oxidation rate (µM/hr)',
                semilog=False
               )

lowess_rate_plot_semilog = lowess_plot(lowess_df_rate, 
                show=True, 
                height=400, 
                width=800, 
                jitter_width=0.3, 
#                 cat_fwd=True, 
                outfile='./plots/figure16_semilog_rate.svg', 
                y_axis_label='max PCA oxidation rate (µM/hr)',
                semilog=True
               )

In [21]:
lowess_df_time = get_lowess_df(
    calibrated_by_time_point, 
    return_param='time', 
    grouping=['date', 'Strain', 'Condition'], 
    cat_fwd=False, 
    thresh=0.75,
    window=24
)

100%|███████████████████████████████████████████| 64/64 [00:02<00:00, 26.13it/s]


In [22]:
lowess_time_plot = lowess_plot(lowess_df_time, 
                show=True, 
                height=400, 
                width=800, 
                jitter_width=0.3, 
#                 cat_fwd=True, 
                outfile='./plots/figure16_TimeToThreshold.svg', 
                y_axis_label='time to 75% of max [PCAred]'
               )

In [24]:
%load_ext watermark
%watermark -v -p numpy,bokeh,pandas,jupyterlab,statsmodels,scipy,dknlab_tools

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.9.15
IPython version      : 8.7.0

numpy       : 1.23.4
bokeh       : 2.3.3
pandas      : 1.5.2
jupyterlab  : 3.5.0
statsmodels : 0.13.2
scipy       : 1.9.3
dknlab_tools: 1.0.3

